In [1]:
import pandas as pd 
import numpy as np 
import requests 
import json

In [2]:
myun = "hpsuresh12345"
mypw = "6a332f5f86503ae3aeec77629dc315264e77a56f"

In [3]:
my_starred_repos = [] 
def get_starred_by_me(): 
    resp_list = [] 
    last_resp = '' 
    first_url_to_get = 'https://api.github.com/user/starred' 
    first_url_resp = requests.get(first_url_to_get, auth=(myun,mypw)) 
    last_resp = first_url_resp 
    resp_list.append(json.loads(first_url_resp.text)) 
     
    while last_resp.links.get('next'): 
        next_url_to_get = last_resp.links['next']['url'] 
        next_url_resp = requests.get(next_url_to_get, auth=(myun,mypw)) 
        last_resp = next_url_resp 
        resp_list.append(json.loads(next_url_resp.text)) 
         
    for i in resp_list: 
        for j in i: 
            msr = j['html_url'] 
            my_starred_repos.append(msr)

In [4]:
get_starred_by_me()

In [5]:
my_starred_repos

[]

In [6]:
my_starred_users = [] 
for ln in my_starred_repos: 
    right_split = ln.split('.com/')[1] 
    starred_usr = right_split.split('/')[0] 
    my_starred_users.append(starred_usr) 
 
my_starred_users

[]

In [7]:
starred_repos = {k:[] for k in set(my_starred_users)} 
def get_starred_by_user(user_name): 
    starred_resp_list = [] 
    last_resp = '' 
    first_url_to_get = 'https://api.github.com/users/'+ user_name +'/starred' 
    first_url_resp = requests.get(first_url_to_get, auth=(myun,mypw)) 
    last_resp = first_url_resp 
    starred_resp_list.append(json.loads(first_url_resp.text)) 
     
    while last_resp.links.get('next'): 
        next_url_to_get = last_resp.links['next']['url'] 
        next_url_resp = requests.get(next_url_to_get, auth=(myun,mypw)) 
        last_resp = next_url_resp 
        starred_resp_list.append(json.loads(next_url_resp.text)) 
         
    for i in starred_resp_list: 
        for j in i: 
            sr = j['html_url'] 
            starred_repos.get(user_name).append(sr)

In [8]:
for usr in list(set(my_starred_users)): 
    print(usr) 
    try: 
        get_starred_by_user(usr) 
    except: 
        print('failed for user', usr)

In [9]:
repo_vocab = [item for sl in list(starred_repos.values()) for item in sl] 

In [10]:
repo_set = list(set(repo_vocab))

In [11]:
len(repo_vocab)

0

In [12]:
all_usr_vector = [] 
for k,v in starred_repos.items(): 
    usr_vector = [] 
    for url in repo_set: 
        if url in v: 
            usr_vector.extend([1]) 
        else: 
            usr_vector.extend([0]) 
    all_usr_vector.append(usr_vector)

In [13]:
df = pd.DataFrame(all_usr_vector, columns=repo_set, index=starred_repos.keys()) 
 
df

""


In [14]:
my_repo_comp = [] 
for i in df.columns: 
    if i in my_starred_repos: 
        my_repo_comp.append(1) 
    else: 
        my_repo_comp.append(0) 
 
mrc = pd.Series(my_repo_comp).to_frame('acombs').T 
 
mrc 

""
acombs


In [15]:
mrc.columns = df.columns 
 
fdf = pd.concat([df, mrc]) 
 
fdf

""
acombs


In [16]:
from scipy.stats import pearsonr 
 
sim_score = {} 
for i in range(len(fdf)): 
    ss = pearsonr(fdf.iloc[-1,:], fdf.iloc[i,:]) 
    sim_score.update({i: ss[0]}) 
 
sf = pd.Series(sim_score).to_frame('similarity') 
sf

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3033: RuntimeWarning: Mean of empty slice.
  mx = x.mean()
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3034: RuntimeWarning: Mean of empty slice.
  my = y.mean()
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


,similarity
0,NaN


In [17]:
sf.sort_values('similarity', ascending=False)

,similarity
0,NaN


In [18]:
fdf.index[6]

IndexError: index 6 is out of bounds for axis 0 with size 1

In [19]:
fdf.iloc[6,:][fdf.iloc[6,:]==1]

IndexError: single positional indexer is out-of-bounds

In [20]:
fdf.index[42]

IndexError: index 42 is out of bounds for axis 0 with size 1

In [21]:
all_recs = fdf.iloc[[6,42,116,159],:] 
all_recs.T

IndexError: positional indexers are out-of-bounds

In [22]:
all_recs[(all_recs==1).all(axis=1)]

NameError: name 'all_recs' is not defined

In [23]:
str_recs_tmp = all_recs[all_recs[myun]==0].copy() 
str_recs = str_recs_tmp.iloc[:,:-1].copy() 
str_recs[(str_recs==1).all(axis=1)]

NameError: name 'all_recs' is not defined

In [24]:
str_recs.sum(axis=1).to_frame('total').sort_values(by='total', ascending=False) 

NameError: name 'str_recs' is not defined